<a id='top'></a>
# Use Client Insights for Wealth Management in meeting preparations

This notebook uses sample data to illustrate the types of results and information available in Client Insights for Wealth Management to assist financial advisors in preparing for an upcoming meeting with an existing client.  

We'll look at preparing to defend the relationship from a high probability client defection to a competing advisor, gaining insights into the client's recent or impending life changes ahead of time, reviewing existing product ownership and identifying the customer's segment or profile.   

This information can assist the advisor in preparing for the meeting by identifying potential new requirements or changes to existing requirements, and by providing a more personal touch through more detailed knowledge of recent events in the client's life or status.

<a id='scenario'></a>
## Scenario
Bruce Smith is a financial advisor servicing high net worth households in the Canadian market whose wealth is typically tied to the mining, natural gas, and petroleum industries. Most of his clients are senior executives working for one of two major large Anglo-Australian natural resources multinationals where Bruce has developed an extensive book of business based on personal referrals. 

Today, Bruce is preparing for a meeting with his longtime client, Alex Anderson, who is relocating his wife Jane and their two children back to the Northwest Territories after a three-year international tour of duty. While the two have stayed in contact, it’s been some time since they’ve met in person.  Bruce wants to make sure he’s well prepared for this meeting in order to re-establish their personal connection and demonstrate the continuing value he’s able to deliver to the Anderson family through highly personalized service and advice to preserve their financial well-being.

While Bruce will typically receive insights to help him prepare for this meeting via an Advisor Desktop or CRM that has been pre-integrated with IBM Watson Client Insight for Wealth Management APIs by his parent firm, let’s take a look at how a Data Scientist or Developer interacting directly with these APIs might help Bruce prepare for his upcoming meeting with Alex and Jane.

## Contents

<a href='#env'> Set Up Environment </a>

<a href='#profile'> Build Client Profile </a>

* <a href='#basic'> Basic Client Information </a>

* <a href='#att'> Client Scores </a>

* <a href='#clep'> Relevant Life Events </a>

* <a href='#segdef'> Segment Description </a>

<a id='env'></a>
## Set up environment


Load key modules that we'll use to keep track of time and interact with the REST API.

In [1]:
import requests
import time
import json
from pandas.io.json import json_normalize
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter, TerminalTrueColorFormatter

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181025132811-0001


 ### Setting up API and URL information

Enter your API key here.  Note that it should be a string, contained in quotes.

In [2]:
api_key = "41e75df9-0d8f-4063-bf72-facad8d1fd7a"
# This is a standard, read-only API key offered to you free of charge for use with this notebook

Next, we set some base variables that we will use extensively in the API calls.  
* base_url - the path to the Client Insights for Wealth Management service on IBM Cloud
* api_context - the type of information we're passing to the API
* head - the headers we're using with the API to provide context and authorization

In [3]:
host = "ci-api.mybluemix.net"
basePath = "/api/v2/wealth/"
base_url = "https://" + host + basePath
api_context = "application/json"
head = {"Authorization" : api_key, 
        "Content-Type" : api_context}

<a id='profile'></a>
## Build the Client Profile

In this section, we'll build a profile for a particular customer. For purposes of illustration, let’s assume that Alex’s customer ID is 5967. We'll retrieve basic information and attrition probability score for Alex and then examine any relevant life events that might affect the Andersons’ financial well-being. Finally, let’s examine the client segmentation into which Alex falls based on his similarity to other clients across Bruce’s book of business. As we go through each element of the profile, comments will discuss potential usage in preparing for our client meeting.

<a href='#basic'> Basic Client Information </a>

<a href='#att'> Client Attrition Score </a>

<a href='#clep'> Relevant Life Events </a>

<a href='#segdef'> Segment Description </a>


In [4]:
# Feel free to try any of the following customer_ids to see different results.
# We have written the commentary for customer_id 1489 only, however.

customer_id = 1489
# customer_id = 1285
# customer_id = 1015
# customer_id = 1725
# customer_id = 1880

page_size = 500

<a id='basic'></a>
### Retrieve Basic Client Information

<a href='#profile'> Back to the Client Profile </a>                  

<a href='#top'> Back to the top </a>

In [5]:
print("Retrieve a specific customer " + str(customer_id)) 

customer_url = "customers/" + str(customer_id)
print("GET customer from: " + base_url + customer_url, end="\r", flush=True) 
customerGet = requests.get(base_url + customer_url,headers=head)

if customerGet.status_code != 200: 
    print('\nAn error occured getting customer') 
    print('customerGet Status Code: ' + str(customerGet.status_code)) 
    print(customerGet.text)

dfbasic = json_normalize(customerGet.json())
print(customerGet)

Retrieve a specific customer 1489
<Response [200]>m: https://ci-api.mybluemix.net/api/v2/wealth/customers/1489


From this information, we can determine key information about our client.  For example, their general location, their place in the household, confirmation that they are currently an active client, their attributed advisor ID (our own, hopefully!) as well as their openness to the use of their data.  

It is always important that we have permission to work with the data provided by a client, with the intent of improving the personalization and customization of the services and advice being provided to them.

In [6]:
print("Location: ", dfbasic.geographic_area_home[0])
print("Head of Household? ", dfbasic.head_of_household_indicator[0])
print("Current Status: ", dfbasic.status[0])
print("Client Since: ", dfbasic.relationship_start_date[0])
print("Last Update: ", dfbasic.effective_date[0])
print("Marketing Opt In: ", dfbasic.advertising_indicator[0])

Location:  Canada MB Manitoba
Head of Household?  True
Current Status:  Active
Client Since:  2016-09-12
Last Update:  2016-09-12
Marketing Opt In:  True


A complete inventory of available fields can be shown below. Note that in some fields, our sample data uses unrealistic placeholder information meant only to show the basic type of information that would be included (e.g., text or a number). 

This is a very small amount of the overall customer profile information. It is meant to ensure consistency with the source system while reducing the personally identifiable information exposed through the API. Our APIs have been designed from the ground up using “privacy by design” principles and contain no more information about individual clients than is necessary to drive analytical results. 

Generally, the application used by the advisor, be that some form of Advisor Desktop or CRM, would receive a more complete history and personal information, including the client’s actual name and identity from the financial institution’s own systems. 


In [7]:
json_str = json.dumps(customerGet.json(), indent=4, sort_keys=True)
print(highlight(json_str, JsonLexer(), TerminalTrueColorFormatter()))

[
    {
        "address_home_city": "Emerald City",
        "address_home_country": "Canada",
        "address_home_state": "MB",
        "address_last_changed_date": "2015-07-17",
        "advertising_indicator": true,
        "aggregation_opt_in_indicator": true,
        "customer_behavior": "Moderate",
        "customer_id": "1489",
        "effective_date": "2016-09-12",
        "geographic_area_home": "Canada MB Manitoba",
        "geographic_area_mailing": "Canada MB Manitoba",
        "geographic_area_work": "Canada MB Manitoba",
        "head_of_household_indicator": true,
        "household_id": "1489",
        "importance_level_code": "Low priority",
        "influence_score": 143843,
        "market_group": "Accumulating",
        "preferred_communication_form": "Email",
        "primary_advisor_id": "1019",
        "primary_branch_proximity": 263.1676044624876,
        "pursuit": "Capital Acquisition",
        "recorded_voice_sample_id": "127199",
        "referrals_value_

<a id='att'></a>
### Retrieve Client Attrition Score

<a href='#profile'> Back to the Client Profile </a>                  

<a href='#top'> Back to the top </a>

In this section we review attrition score for the client. Generally speaking, attrition refers to the likeihood that a given client would leave the firm for another advisor elsewhere. Specifically, we seek to estimate the probability of departure for each client, which can then be used to help monitor the relationship and indicate the need for intervention by the advisor. In preparing for a client meeting, a high score would indicate that the advisor should be prepared to enter into a discussion about the benefits of remaining a client of the firm, dig into sources of issues (see "features" below) and investigate promotions, discounts or offers that might help to retain the client.


In [8]:
# Get the scores for a specific Customer_ID
# Query String parameters; pageSize, page, score_code, effective_date

resp_json = []
page_size = 500
page = 0

print("Retrieve scores for a specific customer " + str(customer_id))

while True:
    scores_url = "customers/" + str(customer_id) + "/scores?page_size=" + str(page_size) + "&page=" + str(page) + "&score_code=ATTRITION&effective_date=2018-09-30"
    print("GET page " + str(page) + " from: " + base_url + scores_url, end="\r", flush=True)
    scoresGet = requests.get(base_url + scores_url,headers=head)
    if scoresGet.json() == [] or scoresGet.status_code != 200 : break
    page = page + 1
    resp_json = resp_json + scoresGet.json()

if scoresGet.status_code != 200:
    print('\nAn error occured getting scores')
    print('scoreGet Status Code: ' + str(scoresGet.status_code))
    print(scoresGet.text)
    
dfscores = json_normalize(resp_json)
json_str = json.dumps(resp_json, indent=4, sort_keys=True)
print("\n")
print(highlight(json_str, JsonLexer(), TerminalTrueColorFormatter()))

Retrieve scores for a specific customer 1489
GET page 1 from: https://ci-api.mybluemix.net/api/v2/wealth/customers/1489/scores?page_size=500&page=1&score_code=ATTRITION&effective_date=2018-09-30

[
    {
        "activity_id": "ATTRITION-2b695ee6-98ef-4753-830e-15ac4e91bc8f",
        "customer_id": "1489",
        "effective_date": "2018-09-30",
        "feature_1_column": "CUSTOMER_MARKET_GROUP_index",
        "feature_1_value": 0.0,
        "feature_2_column": "CUSTOMER_GENDER_index",
        "feature_2_value": 0.0,
        "feature_3_column": "CUSTOMER_ANNUAL_INCOME",
        "feature_3_value": 0.0,
        "model_id": "Attrition",
        "model_scope_forecast_horizon": 1,
        "score_code": "ATTRITION",
        "score_value": 0.0
    },
    {
        "activity_id": "ATTRITION-c62b738b-780e-421f-9888-b103a85f5969",
        "customer_id": "1489",
        "effective_date": "2018-09-30",
        "feature_1_column": "CUSTOMER_MARKET_GROUP_index",
        "feature_1_value": 0.0,
    

In [9]:

print("Focussing on the information, we see the score_code is {sc}.".format(sc = dfscores.score_code[0]))
print("This indicates that the associated score_value of {sv} is the likelihood of client {id}".format(sv = dfscores.score_value[0],
                                                                                                       id = dfscores.customer_id[0]))
print("choosing to leave within the next {fh} month(s), as shown in the forecast_horizon.".format(fh = dfscores.model_scope_forecast_horizon[0]))

Focussing on the information, we see the score_code is ATTRITION.
This indicates that the associated score_value of 0.0 is the likelihood of client 1489
choosing to leave within the next 1 month(s), as shown in the forecast_horizon.


One of the common questions advisors ask is "Why is the likelihood at this level?"  To answer that question, we look at the three main factors (or features) that impact the likelihood for this particular client.

In [10]:
print("We can also see the drivers behind this assessment: \n  {f1} \n  {f2} \n  {f3}".format(f1 = dfscores.feature_1_column[0],
                                                                                              f2 = dfscores.feature_2_column[0],
                                                                                              f3 = dfscores.feature_3_column[0]))

We can also see the drivers behind this assessment: 
  CUSTOMER_MARKET_GROUP_index 
  CUSTOMER_GENDER_index 
  CUSTOMER_ANNUAL_INCOME


It is important that our results are explainable so that advisors find them credible, but also so that they can better understand the drivers of attrition in order to best determine how to respond.  In this case, we see that the Anderson family’s relatively high total assets under management, classification in the “Expatriate Executives” market group, and family size are all characteristics of similar high-risk clients who have departed in the past. 

In preparation for this meeting Bruce is going to want to be sure he has a tight story around his value proposition for higher-net worth clients relative to his competitors, be able to explain how he can best meet the specialized needs of globe-trotting executives and ensure that he is demonstrating how his advice and services incorporate the needs of Alex’s spouse and children. Although Bruce’s primary relationship has been with Alex in the past, he makes a note that Jane is likely to be an important influencer on any decision to remain with Bruce’s practice.


<a id='clep'></a>
### Retrieve Relevant Life Events

Armed with this information, let’s make good on that promise to address the Anderson family’s specific needs and actually deliver on value that helps deflect the possibility of attrition at this crucial time in the relationship. It’s time to examine any potential “money in motion” life events that might influence financial planning topics for the upcoming meeting or affect the timing of advice and recommendations to the Anderson family.

<a href='#profile'> Back to the Client Profile </a>

<a href='#top'> Back to the top </a>

In [11]:
# Get the scores from the trained model and scored customers
# This will return a json document of customers and their scores for all the predictied life events that we scored
# Query String parameters; pageSize, page, score_code, effective_date

print("Retrieve customer's event score data for customer ", customer_id)

resp_json = []
page_size = 500
page = 0

while True:
    scores_url = "customers/" + str(customer_id) + "/event_scores?page_size=" + str(page_size) + "&page=" + str(page) + "&effective_date=2018-09-30"
    print("GET page " + str(page) + " from: " + base_url + scores_url, end="\r", flush=True)
    scoresGet = requests.get(base_url + scores_url,headers=head)
    if scoresGet.json() == [] or scoresGet.status_code != 200 : break
    page = page + 1
    resp_json = resp_json + scoresGet.json()
    
if scoresGet.status_code != 200:
    print('\nAn error occured getting scores')
    print('scoreGet Status Code: ' + str(scoresGet.status_code))
    print(scoresGet.text)

dflep = json_normalize(resp_json)
json_str = json.dumps(resp_json, indent=4, sort_keys=True)
print("\n")
print(highlight(json_str, JsonLexer(), TerminalTrueColorFormatter()))

Retrieve customer's event score data for customer  1489
GET page 1 from: https://ci-api.mybluemix.net/api/v2/wealth/customers/1489/event_scores?page_size=500&page=1&effective_date=2018-09-30

[
    {
        "activity_id": "LIFE_EVENT_PREDICT-b583c2d7-e2b4-4dc8-9aad-8ddb5c3650d9",
        "customer_id": "1489",
        "effective_date": "2018-09-30",
        "event_type_id": "LFE_HOME_PURCHASE",
        "model_id": "LifeEventPredict",
        "model_scope_forecast_horizon": 6,
        "score_code": "LIFE_EVENT_PREDICT",
        "score_value": 0.009523809523809525,
        "segment_id": "1"
    },
    {
        "activity_id": "LIFE_EVENT_PREDICT-b583c2d7-e2b4-4dc8-9aad-8ddb5c3650d9",
        "customer_id": "1489",
        "effective_date": "2018-09-30",
        "event_type_id": "LFE_RELOCATION",
        "model_id": "LifeEventPredict",
        "model_scope_forecast_horizon": 6,
        "score_code": "LIFE_EVENT_PREDICT",
        "score_value": 0.03184713375796178,
        "segment_id": "

Given the Anderson’s pattern of financial transactions, engagement, and other behavior leading up to the current day, we see that Client Insight is predicting both a high probability of a relocation life event and a new home purchase within the next 3 months. Had Bruce not been forewarned and given time to appropriately prepare in advance, he might otherwise have been surprised to learn that the Andersons were still struggling to sell their erstwhile apartment overseas and were living in temporary corporate housing while they search for a new home locally.


Instead, Bruce will enter into that meeting ready to discuss how he can help meet the Andersons’ home financing and short-term liquidity needs with a non-purpose loan that will allow them to close on a new home without having to liquidate securities to raise cash while they are still waiting for their old apartment to sell. This may also make it even less likely that they might transfer their assets to a competing firm, not only because he is able to solve their financial problem elegantly, but because those assets will be used to collateralize the loan. Bruce will be sure to have at the ready the names of his three favorite Real Estate Agents in the Yellowknife area, and the inside scoop on the best and rising school districts.


<a id='segdef'></a>
### Examine the Client's Segment

Finally, let’s look at how the Alex Anderson fits into Bruce’s overall book of business by examining his similarity to Bruce’s other clients. This can be significant because Bruce may wish to prospect for clients similar to his existing ones, or may wish to market to or service different client segments in a more tailored and customized manner specialized to the needs of each segment. 

<a href='#profile'> Back to the Client Profile </a>

<a href='#top'> Back to the top </a>

In [12]:
# Get the scores from the trained model and scored customers
# This will return a json document of customers and their scores for the attrition model that we scored
# Query String parameters; pageSize, page, score_code, effective_date

print("Retrieve segmentation score data")

resp_json = []
page = 0

while True:
    seg_url = "customers/" + str(customer_id) + "/scores?page_size=" + str(page_size) + "&page=" + str(page) + "&score_code=DYNAMIC_SEGMENTATION&effective_date=2018-09-30"
    print("GET page " +str(page) + " from: " + base_url + seg_url, end="\r", flush=True)
    scoresGet = requests.get(base_url + seg_url,headers=head)
    if scoresGet.json() == [] or scoresGet.status_code != 200 : break
    page = page + 1
    resp_json = resp_json + scoresGet.json()
    
if scoresGet.status_code != 200:
    print('An error occured getting scores')
    print('scoreGet Status Code: ' + str(scoresGet.status_code))
    print(scoresGet.text)
    
dfseg = json_normalize(resp_json)
json_str = json.dumps(resp_json, indent=4, sort_keys=True)
print("\n")
print(highlight(json_str, JsonLexer(), TerminalTrueColorFormatter()))

Retrieve segmentation score data
GET page 1 from: https://ci-api.mybluemix.net/api/v2/wealth/customers/1489/scores?page_size=500&page=1&score_code=DYNAMIC_SEGMENTATION&effective_date=2018-09-30

[
    {
        "activity_id": "DYNAMIC_SEGMENTATION-1c9a534c-f509-4ae5-bf38-da926f18df73",
        "customer_id": "1489",
        "effective_date": "2018-09-30",
        "model_id": "DynamicSegmentation",
        "model_scope_forecast_horizon": 0,
        "score_code": "DYNAMIC_SEGMENTATION",
        "segment_id": "2"
    }
]



In [13]:
# Get the segment descriptions (which attributes define the segment) from the trained model
# This will return a json document of segments and their descriptions
# Query String parameters; pageSize, page, score_code, effective_date

print("Retrieve the attributes that define the segments")

resp_json = []
page = 0

while True:
    seg_url = "segments?page_size=" + str(page_size) + "&page=" + str(page) + "&score_code=DYNAMIC_SEGMENTATION&effective_date=2018-09-30"
    print("GET page " + str(page) + " from: " + base_url + seg_url, end="\r", flush=True)
    scoresGet = requests.get(base_url + seg_url,headers=head)
    if scoresGet.json() == [] or scoresGet.status_code != 200 : break
    page = page + 1
    resp_json = resp_json + scoresGet.json()
    
if scoresGet.status_code == 200:
    print('Scores based on query string parameters have been returned')
    print('Number of records: ' + str(len(resp_json)))
else:
    print('An error occured getting scores')
    print('scoreGet Status Code: ' + str(scoresGet.status_code))
    print(scoresGet.text)
    
dfsegdesc = json_normalize(resp_json).sort_values(['segment_id','rank'], ascending=True)
dfsegdesc = dfsegdesc[['segment_id', 'rank', 'column_name', 'max_value', 'min_value']]
#json_str = json.dumps(resp_json, indent=4, sort_keys=True)
#print(highlight(json_str, JsonLexer(), TerminalTrueColorFormatter()))

Retrieve the attributes that define the segments
Scores based on query string parameters have been returnedh/segments?page_size=500&page=1&score_code=DYNAMIC_SEGMENTATION&effective_date=2018-09-30
Number of records: 50


In [14]:
print(dfsegdesc.loc[dfsegdesc.segment_id==dfseg.segment_id[0]])

   segment_id  rank                       column_name  max_value  min_value
30          2     1          CUSTOMER_AGE_RANGE_index        4.0        0.0
31          2     2             CUSTOMER_GENDER_index        1.0        0.0
32          2     3     CUSTOMER_MARITAL_STATUS_index        2.0        0.0
33          2     4         CUSTOMER_URBAN_CODE_index        2.0        0.0
34          2     5  CUSTOMER_EMPLOYMENT_STATUS_index        3.0        0.0


In the above analysis of client with ID 5967, we see that this client belongs to a segment where age is the most important characteristic, followed by shared elements such as:  no more than 2 children, up to 4 active accounts and one or fewer buy/sell trades.  Clearly, within this segment, a discount targeted to active traders would be inappropriate or useless.  

In contrast, Bruce probably has other clients like Alex in this segment for whom the financial well-being of the family is of paramount importance. For this segment, Bruce may want to consider campaigns or events targeted around topics like life insurance, college savings, or generational wealth transfer. Bruce makes a note to mine this segment for opportunities to deepen his relationships with these.

## Summary

You successfully completed this notebook! You have seen the types of results and information available in Client Insights for Wealth Management to assist advisors in preparing for an upcoming meeting with an existing client.

### Authors

**Robert Stanich**, Offering Manager, Client Insights, Watson FSS - <robert.stanich@us.ibm.com>.

**Diane Reynolds**, Chief Data Scientist, Client Insights, Watson FSS - <diane.reynolds@ca.ibm.com>.


<hr>
Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>